In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [2]:
gnn_train_data = pd.read_parquet('../data/curated/ML_data/gnn_train_data.parquet')
gnn_val_data = pd.read_parquet('../data/curated/ML_data/gnn_val_data.parquet')
gnn_test_data = pd.read_parquet('../data/curated/ML_data/gnn_test_data.parquet')

SA2_gnn_data = pd.read_parquet('../data/curated/ML_data/SA2_gnn_data.parquet')
station_inference_gnn_data = pd.read_parquet('../data/curated/ML_data/station_inference_gnn_data.parquet')

station_inference_gnn_data = station_inference_gnn_data.rename({'Station_Na': 'Station Name'}, axis=1)
inference_data = pd.concat([SA2_gnn_data, station_inference_gnn_data], axis=0)
inference_data = inference_data.rename({'Station Name': 'Station_Name'}, axis=1)

In [3]:
# stations_list = [x for x in gnn_train_data['Station_Name'].unique()]
# stations_index = {stations_list[i]:i for i in range(len(stations_list))}
# reverse_stations_index = {v: k for (k, v) in stations_index.items()}

# open npy
station_weights_matrix = np.load('../data/curated/ML_data/station_weights_matrix.npy')
SA2_weights_matrix = np.load('../data/curated/ML_data/station_weights_withSA2_matrix.npy')

with open('../data/curated/ML_features/station_weights_withSA2.json', 'r') as f:
    station_weights_withSA2 = json.load(f)

with open('../data/curated/ML_features/station_weights.json', 'r') as f:
    station_weights = json.load(f)

In [4]:
geospatial_features = ['log_Total_Demand']
non_geospatial_features = ['Weekday', 'PublicHoliday', 'mean_rainfall_value', 'has_school',
       'has_sport_facility', 'has_shopping_centre', 'has_hospital',
       'total_population', ' med_rent_weekly_c2021',
       ' med_mortg_rep_mon_c2021', ' med_person_inc_we_c2021',
       ' med_famly_inc_we_c2021']
label_columns = ['log_Total_Demand']

In [5]:
def DataFactory(raw_dataset, geospatial_features, non_geospatial_features, label_columns, stations_index, inference = False):

    """ Data Factory of GNN """
    
    geospatial_x_batches = []
    non_geospatial_x_batches = []
    y_batches = []
    masks = []

    if inference:
        groupby_column = 'Weekday'
    else:
        groupby_column = 'Business_Date'

    for day, daily_df in tqdm(raw_dataset.groupby([groupby_column])):

        geospatial_x = np.zeros([len(stations_index), len(geospatial_features)])
        y = np.zeros([len(stations_index), len(label_columns)])
        mask = np.zeros([len(stations_index), 1])
        non_geospatial_x = np.zeros([len(stations_index), len(non_geospatial_features)])

        daily_df.set_index('Station_Name', inplace=True)

        for station in daily_df.index:

            geospatial_x[stations_index[station]] = daily_df.loc[station][geospatial_features] # todo inference. 
            if not inference:
                y[stations_index[station]] = daily_df.loc[station][label_columns]
            mask[stations_index[station]] = 1
            non_geospatial_x[stations_index[station]] = daily_df.loc[station][non_geospatial_features]
                
        geospatial_x_batches.append(geospatial_x)
        y_batches.append(y)
        masks.append(mask.flatten())

        non_geospatial_x_batches.append(non_geospatial_x)

        
    return geospatial_x_batches, non_geospatial_x_batches, y_batches, masks

In [6]:
train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks = DataFactory(gnn_train_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks = DataFactory(gnn_val_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks = DataFactory(gnn_test_data, geospatial_features, non_geospatial_features, label_columns, station_weights)

# inference_geospatial_X_batches, inference_non_geospatial_X_batches, inference_y_batches, inference_masks = DataFactory(inference_data, geospatial_features, non_geospatial_features, label_columns, station_weights_withSA2, inference = True)

100%|██████████| 82/82 [00:16<00:00,  5.07it/s]


In [7]:
import sys
import os

py_file_location = '../'
home_directory = '../'

sys.path.append(os.path.abspath(py_file_location))
from model.model_class.environment import *

In [8]:
from model.model_class import GNN

In [9]:
SEED = 42

In [10]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 1
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 382/382 [00:17<00:00, 21.50it/s]


 Epoch 1 Train | Loss:  0.0871 | R2:  0.9055| MSE:  0.0867 | RMSE:  0.2944 | MAE:  0.1938 


100%|██████████| 82/82 [00:01<00:00, 47.88it/s]
/Users/tg.chenny/Desktop/1. University/2. Masters/9. Geospatial Data Analysis/A4/GEOM90006-A4_TrainDemandInGreaterMelbourne__Groupwork__Py/model/model_class/__template__.py:193: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  val_y_tensor = torch.FloatTensor(val_y).to(self.device)


Epoch 1 Val | Loss:  0.0148 | R2:  0.9852| MSE:  0.0148 | RMSE:  0.1215 | MAE:  0.0969 


100%|██████████| 382/382 [00:15<00:00, 24.82it/s]


...

Epoch 11 Val | Loss:  0.0035 | R2:  0.9965| MSE:  0.0035 | RMSE:  0.0589 | MAE:  0.0446 





100%|██████████| 82/82 [00:00<00:00, 108.01it/s]


Epoch 8 Val | Loss:  0.0020 | R2:  0.9979| MSE:  0.0020 | RMSE:  0.0452 | MAE:  0.0315 


100%|██████████| 82/82 [00:00<00:00, 107.88it/s]


Epoch 8 Val | Loss:  0.0021 | R2:  0.9977| MSE:  0.0021 | RMSE:  0.0456 | MAE:  0.0324 


In [15]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 2
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [00:20<00:00, 18.83it/s]


 Epoch 1 Train | Loss:  0.1123 | R2:  0.8780| MSE:  0.1119 | RMSE:  0.3345 | MAE:  0.2205 


100%|██████████| 82/82 [00:01<00:00, 66.16it/s]


Epoch 1 Val | Loss:  0.0171 | R2:  0.9829| MSE:  0.0171 | RMSE:  0.1308 | MAE:  0.1019 


...

Epoch 11 Val | Loss:  0.0071 | R2:  0.9929| MSE:  0.0071 | RMSE:  0.0841 | MAE:  0.0687 





100%|██████████| 82/82 [00:01<00:00, 70.35it/s]


Epoch 10 Val | Loss:  0.0062 | R2:  0.9938| MSE:  0.0062 | RMSE:  0.0788 | MAE:  0.0682 


100%|██████████| 82/82 [00:01<00:00, 72.08it/s]


Epoch 10 Val | Loss:  0.0062 | R2:  0.9931| MSE:  0.0062 | RMSE:  0.0786 | MAE:  0.0685 


In [16]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 3
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

100%|██████████| 382/382 [00:29<00:00, 13.05it/s]


 Epoch 1 Train | Loss:  0.1345 | R2:  0.8540| MSE:  0.1339 | RMSE:  0.3660 | MAE:  0.2439 


100%|██████████| 82/82 [00:01<00:00, 46.28it/s]


Epoch 1 Val | Loss:  0.0384 | R2:  0.9615| MSE:  0.0384 | RMSE:  0.1960 | MAE:  0.1198 


100%|██████████| 382/382 [00:29<00:00, 12.79it/s]


...

Epoch 10 Val | Loss:  0.0062 | R2:  0.9938| MSE:  0.0062 | RMSE:  0.0787 | MAE:  0.0485 





100%|██████████| 82/82 [00:02<00:00, 34.44it/s]


Epoch 8 Val | Loss:  0.0032 | R2:  0.9968| MSE:  0.0032 | RMSE:  0.0561 | MAE:  0.0411 


100%|██████████| 82/82 [00:02<00:00, 35.68it/s]


Epoch 8 Val | Loss:  0.0031 | R2:  0.9966| MSE:  0.0031 | RMSE:  0.0556 | MAE:  0.0413 


In [14]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0
    n_gnn_layers = 4
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

100%|██████████| 382/382 [00:49<00:00,  7.78it/s]


 Epoch 1 Train | Loss:  0.1500 | R2:  0.8370| MSE:  0.1495 | RMSE:  0.3867 | MAE:  0.2570 


100%|██████████| 82/82 [00:02<00:00, 28.88it/s]


Epoch 1 Val | Loss:  0.0254 | R2:  0.9745| MSE:  0.0254 | RMSE:  0.1595 | MAE:  0.1151 


100%|██████████| 382/382 [02:09<00:00,  2.94it/s]


...

Epoch 8 Val | Loss:  0.0070 | R2:  0.9930| MSE:  0.0070 | RMSE:  0.0835 | MAE:  0.0522 





100%|██████████| 82/82 [00:02<00:00, 35.49it/s]


Epoch 4 Val | Loss:  0.0057 | R2:  0.9943| MSE:  0.0057 | RMSE:  0.0753 | MAE:  0.0526 


100%|██████████| 82/82 [00:02<00:00, 35.24it/s]

Epoch 4 Val | Loss:  0.0056 | R2:  0.9938| MSE:  0.0056 | RMSE:  0.0748 | MAE:  0.0541 
